<a href="https://colab.research.google.com/github/racoope70/daytrading-with-ml/blob/main/lightgbm_enhanced_strategy_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get remove --purge -y cuda* libcuda* nvidia* || echo "No conflicting CUDA packages"
!apt-get autoremove -y
!apt-get clean

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'cuda-toolkit-12-4-config-common' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-11-0' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-11-1' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-11-7' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-11-8' for glob 'cuda*'
Note, selecting 'cuda-toolkit-12-5-config-common' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-0' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-1' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-2' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-3' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-4' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-5' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-6' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-8' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-9' for glob 'cuda*'
Note, selecting 'cuda-toolkit-12-6-config

In [2]:
#Protocol Buffer Fix (for TensorFlow)
!pip uninstall -y protobuf
!pip install protobuf==3.20.3

Found existing installation: protobuf 5.29.5
Uninstalling protobuf-5.29.5:
  Successfully uninstalled protobuf-5.29.5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 10.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 3.20.3 which is incompatible.
tensorflow-metadata 1.17.1 requires protobuf<6.0.0,>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.
ydf 0.12.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 3.20.3 which is incompatible.


In [3]:
#Update Colab Environment and System Libraries
!apt-get update -y && apt-get upgrade -y


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,253 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,050 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 P

In [4]:
#Install Correct Version of CUDA for Colab GPU
!apt-get update -qq && apt-get install -y \
    libcusolver11 libcusparse11 libcurand10 libcufft10 libnppig10 libnppc10 libnppial10 \
    cuda-toolkit-12-4

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package libnppig10
E: Unable to locate package libnppc10
E: Unable to locate package libnppial10


In [5]:
#Set Correct CUDA Paths
import os
os.environ['CUDA_HOME'] = '/usr/local/cuda-12.4'
os.environ['PATH'] += ':/usr/local/cuda-12.4/bin'
os.environ['LD_LIBRARY_PATH'] += ':/usr/local/cuda-12.4/lib64'


In [6]:
#Install RAPIDS and NVIDIA Dependencies
!pip install --extra-index-url=https://pypi.nvidia.com \
    cuml-cu12==25.2.0 cudf-cu12==25.2.0 cupy-cuda12x dask-cuda==25.2.0 dask-cudf-cu12==25.2.0


Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 70.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 137.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 47.2 MB/s eta 0:00:00
  Attempting uninstall: cudf-cu12
    Found existing installation: cudf-cu12 25.2.1
    Uninstalling cudf-cu12-25.2.1:
      Successfully uninstalled cudf-cu12-25.2.1
  Attempting uninstall: dask-cudf-cu12
    Found existing installation: dask-cudf-cu12 25.2.2
    Uninstalling dask-cudf-cu12-25.2.2:
      Successfully uninstalled dask-cudf-cu12-25.2.2
  Attempting uninstall: cuml-cu12
    Found existing installation: cuml-cu12 25.2.1
    Uninstalling cuml-cu12-25.2.1:
      Successfully uninstalled cuml-cu12-25.2.1


In [7]:
#Install TensorFlow (latest GPU-compatible version)
!pip install tensorflow==2.18.0

#Install Stable Baselines3 and Trading Libraries
!pip install stable-baselines3[extra] gymnasium gym-anytrading yfinance xgboost joblib

#Install Miscellaneous Libraries
!pip install matplotlib scikit-learn pandas numba==0.61.0

#Install PyTorch with GPU Support
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.2/172.2 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 122.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 101.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.5/184.5 kB 17.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink

In [8]:
#Install TensorFlow (latest GPU-compatible version)
!pip install tensorflow==2.18.0


import tensorflow as tf

gpus = tf.config.list_physical_devices("GPU")
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("TensorFlow GPU memory growth enabled")
    except RuntimeError as e:
        print(f"TensorFlow GPU memory config failed: {e}")


TensorFlow GPU memory growth enabled


In [9]:
!pip install stable-baselines3[extra] gymnasium gym-anytrading yfinance --quiet
!pip install stable-baselines3[extra] --quiet


In [10]:
!rm -rf /content/drive

In [11]:
from google.colab import drive
import os
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [12]:
# === Imports ===
import os
import gc
import json
import time
import random
import joblib
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

from datetime import datetime
from collections import deque

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from cuml.ensemble import RandomForestClassifier
import cupy as cp
import cudf
import cuml
import dask
import numba

import gymnasium as gym
import gym_anytrading
from gym_anytrading.envs import StocksEnv
from gymnasium.spaces import Box

from stable_baselines3 import PPO, SAC
from stable_baselines3.common.noise import NormalActionNoise
from stable_baselines3.common.vec_env import DummyVecEnv

# === Ticker List and Config ===
ticker_list = [
    'AAPL', 'TSLA', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'META', 'BRK-B', 'JPM', 'JNJ',
    'XOM', 'V', 'PG', 'UNH', 'MA', 'HD', 'LLY', 'MRK', 'PEP', 'KO',
    'BAC', 'ABBV', 'AVGO', 'PFE', 'COST', 'CSCO', 'TMO', 'ABT', 'ACN', 'WMT',
    'MCD', 'ADBE', 'DHR', 'CRM', 'NKE', 'INTC', 'QCOM', 'NEE', 'AMD', 'TXN',
    'AMGN', 'UPS', 'LIN', 'PM', 'UNP', 'BMY', 'LOW', 'RTX', 'CVX', 'IBM',
    'GE', 'SBUX', 'ORCL'
]

strategy_name = "sac_ppo_td3_multi_stock_v1"

CONFIG = {
    'symbols': [],
    'period': '720d',
    'interval': '1h',
    'target': 'Target',
    'sharpe_threshold': 1.5,
    'return_threshold': 1.25,
    'strategy_name': strategy_name
}

# === Download Function ===
def download_stock_data(ticker, period="720d", interval="1h", max_retries=5):
    for attempt in range(1, max_retries + 1):
        try:
            print(f"Attempt {attempt}: Downloading {ticker}...")
            df = yf.download(ticker, period=period, interval=interval)
            if not df.empty:
                df.reset_index(inplace=True)
                df['Symbol'] = ticker
                return df
            raise ValueError("Empty data")
        except Exception as e:
            print(f"Error: {e}. Retrying in {attempt * 5} sec...")
            time.sleep(attempt * 5)
    print(f"Failed to download {ticker}")
    return None

# === Feature Engineering Function ===
def compute_enhanced_features(df):
    df = df.copy()
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)
    df = df.loc[:, ~df.columns.duplicated()]

    df['SMA_20'] = df['Close'].rolling(20).mean()
    df['STD_20'] = df['Close'].rolling(20).std()
    df['Upper_Band'] = df['SMA_20'] + 2 * df['STD_20']
    df['Lower_Band'] = df['SMA_20'] - 2 * df['STD_20']
    df['Lowest_Low'] = df['Low'].rolling(14).min()
    df['Highest_High'] = df['High'].rolling(14).max()
    denom = (df['Highest_High'] - df['Lowest_Low']).replace(0, np.nan)
    df['Stoch'] = ((df['Close'] - df['Lowest_Low']) / denom) * 100
    df['ROC'] = df['Close'].pct_change(10)
    df['OBV'] = (np.sign(df['Close'].diff()) * df['Volume']).cumsum()
    typical_price = (df['High'] + df['Low'] + df['Close']) / 3
    df['CCI'] = (typical_price - typical_price.rolling(20).mean()) / (0.015 * typical_price.rolling(20).std())
    df['PROC'] = ((df['Close'] - df['Close'].shift(12)) / df['Close'].shift(12)) * 100
    df['Rolling_Mean_50'] = df['Close'].rolling(50).mean()
    df['Expanding_Mean'] = df['Close'].expanding().mean()
    df['EMA_10'] = df['Close'].ewm(span=10).mean()
    df['EMA_50'] = df['Close'].ewm(span=50).mean()
    df['MACD_Line'] = df['Close'].ewm(span=12).mean() - df['Close'].ewm(span=26).mean()
    df['MACD_Signal'] = df['MACD_Line'].ewm(span=9).mean()
    df['MACD_Hist'] = df['MACD_Line'] - df['MACD_Signal']
    delta = df['Close'].diff()
    gain = delta.where(delta > 0, 0).rolling(14).mean()
    loss = -delta.where(delta < 0, 0).rolling(14).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))
    df['True_Range'] = df[['High', 'Low', 'Close']].apply(
        lambda x: max(x.iloc[0] - x.iloc[1], abs(x.iloc[0] - x.iloc[2]), abs(x.iloc[1] - x.iloc[2])), axis=1)
    df['ATR'] = df['True_Range'].rolling(14).mean()
    df['+DM'] = np.where((df['High'].diff() > df['Low'].diff()) & (df['High'].diff() > 0), df['High'].diff(), 0)
    df['-DM'] = np.where((df['Low'].diff() > df['High'].diff()) & (df['Low'].diff() > 0), df['Low'].diff(), 0)
    df['+DI'] = 100 * df['+DM'].rolling(14).mean() / df['ATR']
    df['-DI'] = 100 * df['-DM'].rolling(14).mean() / df['ATR']
    df['ADX'] = abs(df['+DI'] - df['-DI']).rolling(14).mean()
    df['Volume_Avg'] = df['Volume'].rolling(20).mean()
    df['Volume_Change'] = df['Volume'].pct_change()
    df['Volume_Change_MA'] = df['Volume_Change'].rolling(10).mean()
    df['Volume_Change_Ratio'] = df['Volume_Change'] / df['Volume_Change'].shift(1)
    df['Relative_Volume'] = df['Volume'] / df['Volume_Avg']
    df['Trailing_Stop'] = np.minimum(df['Close'] * 0.985, df['Close'] - (df['ATR'] * 0.3))
    df['Buy_Signal'] = np.where((df['RSI'] < 60) & (df['EMA_10'] > df['EMA_50']) &
                                ((df['MACD_Line'] > df['MACD_Signal']) | (df['MACD_Line'].diff() > 0)) &
                                (df['Volume'] > (0.4 * df['Volume_Avg'])) & (df['ADX'] > 18), 1, 0)
    df['Sell_Signal'] = np.where(((df['EMA_10'] < df['EMA_50']) & (df['RSI'] > 60)) |
                                 ((df['MACD_Line'] < df['MACD_Signal']) & (df['RSI'] > 65)) |
                                 (df['Close'] < df['Trailing_Stop']) |
                                 ((df['Volume'] > 0.5 * df['Volume_Avg']) & (df['ADX'] > 20)), 1, 0)
    df['Sell_Signal_Debug'] = np.where(((df['MACD_Hist'] < 0.5) | (df['MACD_Line'] < df['MACD_Signal'])) &
                                       (df['RSI'] < 55) & (df['ADX'] > 15) &
                                       ((df['Close'] < df['Trailing_Stop']) | (df['EMA_10'] < df['EMA_50'])) &
                                       (df['Volume'] > 0.5 * df['Volume_Avg']), 1, 0)
    df['Future_Close'] = df['Close'].shift(-10)
    df['Volatility'] = df['Close'].pct_change().rolling(window=20).std()
    df['Return'] = (df['Future_Close'] - df['Close']) / df['Close']
    df['Target'] = np.select([df['Return'] > 0.02, df['Return'] < -0.02], [1, -1], default=0)
    df['Multi_Class_Target'] = df['Target']
    df['Hour'] = pd.to_datetime(df['Datetime']).dt.hour
    df['DayOfWeek'] = pd.to_datetime(df['Datetime']).dt.dayofweek
    df['Session'] = np.where((df['Hour'] >= 9) & (df['Hour'] <= 16), 'Regular',
                             np.where((df['Hour'] < 9), 'Pre-market', 'After-hours'))
    df['MACD_Crossover'] = np.where(df['MACD_Line'] > df['MACD_Signal'], 1, 0)
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.dropna(inplace=True)
    return df

# === Main Pipeline ===
all_dfs = []

for ticker in ticker_list:
    df_single = download_stock_data(ticker, period=CONFIG['period'], interval=CONFIG['interval'])
    if df_single is not None:
        try:
            df_features = compute_enhanced_features(df_single)
            all_dfs.append(df_features)
        except Exception as e:
            print(f"Feature engineering failed for {ticker}: {e}")
    else:
        print(f"Failed to download {ticker}")

if all_dfs:
    df = pd.concat(all_dfs, ignore_index=True)
    print(f"Combined dataset created with shape: {df.shape}")
else:
    df = pd.DataFrame()
    print("No data available.")

if not df.empty:
    df.to_csv("multi_stock_feature_engineered_dataset.csv", index=False)
    print("Saved locally to multi_stock_feature_engineered_dataset.csv")

    drive_path = "/content/drive/MyDrive/trading_data/"
    os.makedirs(drive_path, exist_ok=True)
    df.to_csv(os.path.join(drive_path, "multi_stock_feature_engineered_dataset.csv"), index=False)
    print(f"Also saved to Google Drive at {drive_path}multi_stock_feature_engineered_dataset.csv")


Attempt 1: Downloading AAPL...


/tmp/ipython-input-12-1265363067.py:64: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period=period, interval=interval)
[*********************100%***********************]  1 of 1 completed


Attempt 1: Downloading TSLA...


/tmp/ipython-input-12-1265363067.py:64: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period=period, interval=interval)
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-12-1265363067.py:64: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period=period, interval=interval)


Attempt 1: Downloading MSFT...


...

Attempt 1: Downloading SBUX...


/tmp/ipython-input-12-1265363067.py:64: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period=period, interval=interval)
[*********************100%***********************]  1 of 1 completed


Attempt 1: Downloading ORCL...


/tmp/ipython-input-12-1265363067.py:64: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period=period, interval=interval)
[*********************100%***********************]  1 of 1 completed


Combined dataset created with shape: (262327, 51)
Saved locally to multi_stock_feature_engineered_dataset.csv
Also saved to Google Drive at /content/drive/MyDrive/trading_data/multi_stock_feature_engineered_dataset.csv


In [17]:
#added slippage
# === Imports ===
import os
import gc
import time
import json
import pickle
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from lightgbm import LGBMClassifier
from lightgbm import early_stopping, log_evaluation

# === Paths ===
SAVE_DIR = "/content/drive/MyDrive/QuantConnect/results_lightgbm/lightgbm_walkforward_models"
RESULTS_DIR = "/content/drive/MyDrive/QuantConnect/results_lightgbm"
os.makedirs(SAVE_DIR, exist_ok=True)
os.makedirs(os.path.join(RESULTS_DIR, "plots"), exist_ok=True)
os.makedirs(os.path.join(RESULTS_DIR, "data"), exist_ok=True)

# === Configuration ===
TEST_MODE = True
TICKERS = ['AAPL'] if TEST_MODE else [  # Replace with full list for production run
    'AAPL', 'TSLA', 'MSFT', 'GOOGL', 'AMZN'
]
features_list = ["SMA_50", "EMA_20", "RSI", "MACD", "Signal_Line", "ATR", "OBV", "CCI"]
SLIPPAGE_RATE = 0.001  # 0.1% slippage per trade

# === Feature Engineering ===
def compute_technical_indicators(df):
    df['SMA_50'] = df['Close'].rolling(window=50).mean()
    df['EMA_20'] = df['Close'].ewm(span=20).mean()
    delta = df['Close'].diff()
    gain = delta.where(delta > 0, 0).rolling(window=14).mean()
    loss = -delta.where(delta < 0, 0).rolling(window=14).mean()
    rs = gain / (loss + 1e-6)
    df['RSI'] = 100 - (100 / (1 + rs))
    df['MACD'] = df['Close'].ewm(span=12).mean() - df['Close'].ewm(span=26).mean()
    df['Signal_Line'] = df['MACD'].ewm(span=9).mean()
    df['ATR'] = df['High'].rolling(window=14).max() - df['Low'].rolling(window=14).min()
    df['OBV'] = (np.sign(df['Close'].diff()) * df['Volume']).fillna(0).cumsum()
    tp = (df['High'] + df['Low'] + df['Close']) / 3
    df['CCI'] = (tp - tp.rolling(20).mean()) / (0.015 * tp.rolling(20).std())
    df.dropna(inplace=True)
    return df

# === Label Generation ===
def generate_labels(df):
    df['Target'] = (df['Close'].shift(-1) > df['Close']).astype(int)
    return df.dropna()

# === Save Scaler and Feature Metadata ===
def save_scaler_and_features(df, ticker):
    scaler = MinMaxScaler()
    scaler.fit(df[features_list])
    with open(os.path.join(SAVE_DIR, f"scaler_{ticker}.pkl"), "wb") as f:
        pickle.dump(scaler, f)
    with open(os.path.join(SAVE_DIR, f"features_{ticker}.json"), "w") as f:
        json.dump(features_list, f)
    print(f"✅ Saved scaler and features for {ticker}")

# === Walkforward Training + Evaluation ===
def run_lgbm_walkforward(ticker, window_size=3000, step_size=500, initial_cash=100000):
    print(f"\n📊 Running walkforward LGBM for {ticker}")
    df = yf.download(ticker, period="720d", interval="1h", progress=False)
    if df.empty:
        print("⚠️ Empty dataframe. Skipping.")
        return

    df = compute_technical_indicators(df)
    df = generate_labels(df)
    if df.empty or not all(col in df.columns for col in features_list):
        print("⚠️ Missing required columns after feature engineering.")
        return

    save_scaler_and_features(df, ticker)

    scaler = MinMaxScaler()
    portfolio_lgbm, portfolio_hold, y_pred_all, y_true_all = [], [], [], []
    capital = initial_cash
    shares = initial_cash / df['Close'].iloc[0]

    for start in range(0, len(df) - window_size, step_size):
        segment = df.iloc[start:start+window_size].copy()
        if segment.shape[0] < window_size:
            break

        X = segment[features_list]
        y = segment['Target']
        X_scaled = scaler.fit_transform(X)
        X_train, X_test = X_scaled[:-step_size], X_scaled[-step_size:]
        y_train, y_test = y[:-step_size], y[-step_size:]

        model = LGBMClassifier(n_estimators=500, learning_rate=0.05, random_state=42)
        model.fit(X_train, y_train,
                  eval_set=[(X_test, y_test)],
                  callbacks=[early_stopping(20), log_evaluation(0)])

        preds = (model.predict(X_test) > 0.5).astype(int)
        prices = segment['Close'].values[-step_size:]

        if len(preds) != step_size or len(prices) != step_size:
            continue

        y_pred_all.extend(preds)
        y_true_all.extend(y_test.tolist())

        for i, pred in enumerate(preds):
            raw_price = prices[i]

            if pred:  # BUY
                exec_price = raw_price * (1 + SLIPPAGE_RATE)
                capital = shares * exec_price
            else:  # SELL
                exec_price = raw_price * (1 - SLIPPAGE_RATE)
                shares = capital / exec_price

            portfolio_lgbm.append(float(capital))
            portfolio_hold.append(float(shares * raw_price))

    if len(y_pred_all) < 2:
        print(f"⚠️ Skipping {ticker} — not enough data for evaluation.")
        return

    y_pred_all = np.array(y_pred_all[:len(y_true_all)])
    y_true_all = np.array(y_true_all[:len(y_pred_all)])

    accuracy = accuracy_score(y_true_all, y_pred_all)
    precision = precision_score(y_true_all, y_pred_all)
    recall = recall_score(y_true_all, y_pred_all)
    f1 = f1_score(y_true_all, y_pred_all)
    returns = np.diff(portfolio_lgbm) / (np.array(portfolio_lgbm[:-1]) + 1e-6)
    sharpe = np.mean(returns) / (np.std(returns) + 1e-6) * np.sqrt(252)
    drawdown = np.max(np.maximum.accumulate(portfolio_lgbm) - portfolio_lgbm)

    final_portfolio = portfolio_lgbm[-1]
    final_hold = portfolio_hold[-1]

    print(f"✅ {ticker} | Final: {final_portfolio:.2f} | Buy&Hold: {final_hold:.2f} | Sharpe: {sharpe:.3f} | Accuracy: {accuracy:.3f}")

    model.booster_.save_model(f"{SAVE_DIR}/model_{ticker}.txt")

    metrics_df = pd.DataFrame([{
        "Ticker": ticker,
        "Final_Portfolio": round(final_portfolio, 2),
        "Final_Hold": round(final_hold, 2),
        "Return_%": round((final_portfolio - initial_cash) / initial_cash * 100, 2),
        "Return_Hold_%": round((final_hold - initial_cash) / initial_cash * 100, 2),
        "Sharpe": round(sharpe, 4),
        "Accuracy": round(accuracy, 4),
        "Precision": round(precision, 4),
        "Recall": round(recall, 4),
        "F1_Score": round(f1, 4),
        "Drawdown": round(drawdown, 2)
    }])
    metrics_df.to_csv(f"{RESULTS_DIR}/metrics_{ticker}.csv", index=False)

    summary_path = os.path.join(RESULTS_DIR, "lightgbm_walkforward_summary.csv")
    if os.path.exists(summary_path):
        all_metrics = pd.read_csv(summary_path)
        all_metrics = all_metrics[all_metrics["Ticker"] != ticker]
        all_metrics = pd.concat([all_metrics, metrics_df], ignore_index=True)
    else:
        all_metrics = metrics_df
    all_metrics.to_csv(summary_path, index=False)

    plt.figure(figsize=(12, 6))
    plt.plot(portfolio_lgbm, label="LGBM Strategy")
    plt.plot(portfolio_hold, label="Buy & Hold")
    plt.title(f"{ticker} Portfolio Value")
    plt.xlabel("Step")
    plt.ylabel("Portfolio Value")
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"{RESULTS_DIR}/plots/{ticker}_portfolio_plot.png")
    plt.close()

    print(f"📁 Saved all artifacts for {ticker}")

# === Run All Tickers ===
for ticker in TICKERS:
    run_lgbm_walkforward(ticker)
    gc.collect()
    time.sleep(1)



📊 Running walkforward LGBM for AAPL


/tmp/ipython-input-17-858394504.py:71: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period="720d", interval="1h", progress=False)


✅ Saved scaler and features for AAPL
[LightGBM] [Info] Number of positive: 1286, number of negative: 1214
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000200 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2040
[LightGBM] [Info] Number of data points in the train set: 2500, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.514400 -> initscore=0.057616
[LightGBM] [Info] Start training from score 0.057616
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[10]	valid_0's binary_logloss: 0.691907
[LightGBM] [Info] Number of positive: 1297, number of negative: 1203
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000197 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2038
[LightGBM] [Info] Number of data points in the train set: 2500, number of 

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/tmp/ipython-input-17-858394504.py:120: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  portfolio_lgbm.append(float(capital))
/tmp/ipython-input-17-858394504.py:121: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  portfolio_hold.append(float(shares * price))
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and wil

[LightGBM] [Info] Number of positive: 1308, number of negative: 1192
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000246 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2040
[LightGBM] [Info] Number of data points in the train set: 2500, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.523200 -> initscore=0.092867
[LightGBM] [Info] Start training from score 0.092867
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[2]	valid_0's binary_logloss: 0.691656
[LightGBM] [Info] Number of positive: 1296, number of negative: 1204
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000195 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2040
[LightGBM] [Info] Number of data points in the train set: 2500, number of used features: 8
[LightGBM] [Info] [bi

In [20]:
#Add Live Updating / Online Learning
!pip install yfinance lightgbm scikit-learn matplotlib


In [21]:
# === Imports ===
import os
import gc
import time
import json
import pickle
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from lightgbm import LGBMClassifier
from lightgbm import early_stopping, log_evaluation

# === Paths ===
SAVE_DIR = "/content/drive/MyDrive/QuantConnect/results_lightgbm/lightgbm_walkforward_models"
RESULTS_DIR = "/content/drive/MyDrive/QuantConnect/results_lightgbm"
os.makedirs(SAVE_DIR, exist_ok=True)
os.makedirs(os.path.join(RESULTS_DIR, "plots"), exist_ok=True)
os.makedirs(os.path.join(RESULTS_DIR, "data"), exist_ok=True)

# === Configuration ===
TEST_MODE = True
TICKERS = ['AAPL'] if TEST_MODE else [
    'AAPL', 'TSLA', 'MSFT', 'GOOGL', 'AMZN'
]
features_list = ["SMA_50", "EMA_20", "RSI", "MACD", "Signal_Line", "ATR", "OBV", "CCI"]
SLIPPAGE_RATE = 0.001

# === Feature Engineering ===
def compute_technical_indicators(df):
    df['SMA_50'] = df['Close'].rolling(window=50).mean()
    df['EMA_20'] = df['Close'].ewm(span=20).mean()
    delta = df['Close'].diff()
    gain = delta.where(delta > 0, 0).rolling(window=14).mean()
    loss = -delta.where(delta < 0, 0).rolling(window=14).mean()
    rs = gain / (loss + 1e-6)
    df['RSI'] = 100 - (100 / (1 + rs))
    df['MACD'] = df['Close'].ewm(span=12).mean() - df['Close'].ewm(span=26).mean()
    df['Signal_Line'] = df['MACD'].ewm(span=9).mean()
    df['ATR'] = df['High'].rolling(window=14).max() - df['Low'].rolling(window=14).min()
    df['OBV'] = (np.sign(df['Close'].diff()) * df['Volume']).fillna(0).cumsum()
    tp = (df['High'] + df['Low'] + df['Close']) / 3
    df['CCI'] = (tp - tp.rolling(20).mean()) / (0.015 * tp.rolling(20).std())
    df.dropna(inplace=True)
    return df

# === Label Generation ===
def generate_labels(df):
    df['Target'] = (df['Close'].shift(-1) > df['Close']).astype(int)
    return df.dropna()

# === Save Scaler and Feature Metadata ===
def save_scaler_and_features(df, ticker):
    scaler = MinMaxScaler()
    scaler.fit(df[features_list])
    with open(os.path.join(SAVE_DIR, f"scaler_{ticker}.pkl"), "wb") as f:
        pickle.dump(scaler, f)
    with open(os.path.join(SAVE_DIR, f"features_{ticker}.json"), "w") as f:
        json.dump(features_list, f)
    print(f"✅ Saved scaler and features for {ticker}")

# === Walkforward Training + Evaluation ===
def run_lgbm_walkforward(ticker, window_size=3000, step_size=500, initial_cash=100000):
    print(f"\n📊 Running walkforward LGBM for {ticker}")
    df = yf.download(ticker, period="720d", interval="1h", progress=False)
    if df.empty:
        print("⚠️ Empty dataframe. Skipping.")
        return

    df = compute_technical_indicators(df)
    df = generate_labels(df)
    if df.empty or not all(col in df.columns for col in features_list):
        print("⚠️ Missing required columns after feature engineering.")
        return

    save_scaler_and_features(df, ticker)

    scaler = MinMaxScaler()
    portfolio_lgbm, portfolio_hold, y_pred_all, y_true_all = [], [], [], []
    capital = initial_cash
    shares = initial_cash / df['Close'].iloc[0]

    model = LGBMClassifier(n_estimators=100, learning_rate=0.05, warm_start=True, random_state=42)
    model_initialized = False

    for start in range(0, len(df) - window_size, step_size):
        segment = df.iloc[start:start+window_size].copy()
        if segment.shape[0] < window_size:
            break

        X = segment[features_list]
        y = segment['Target']
        X_scaled = scaler.fit_transform(X)
        X_train, X_test = X_scaled[:-step_size], X_scaled[-step_size:]
        y_train, y_test = y[:-step_size], y[-step_size:]

        if not model_initialized:
            model.fit(X_train, y_train,
                      eval_set=[(X_test, y_test)],
                      callbacks=[early_stopping(20), log_evaluation(0)])
            model_initialized = True
        else:
            model.n_estimators += 100
            model.fit(X_train, y_train,
                      eval_set=[(X_test, y_test)],
                      callbacks=[early_stopping(20), log_evaluation(0)],
                      init_model=model)

        preds = (model.predict(X_test) > 0.5).astype(int)
        prices = segment['Close'].values[-step_size:]

        if len(preds) != step_size or len(prices) != step_size:
            continue

        y_pred_all.extend(preds)
        y_true_all.extend(y_test.tolist())

        # === Regime shift / reset trigger ===
        acc_window = accuracy_score(y_test, preds)
        if acc_window < 0.5:
            print("⚠️ Accuracy dropped below threshold. Resetting model.")
            model = LGBMClassifier(n_estimators=100, learning_rate=0.05, warm_start=True, random_state=42)
            model_initialized = False

        # === Trade Simulation with Slippage ===
        for i, pred in enumerate(preds):
            raw_price = prices[i]
            if pred:  # BUY
                exec_price = raw_price * (1 + SLIPPAGE_RATE)
                capital = shares * exec_price
            else:  # SELL
                exec_price = raw_price * (1 - SLIPPAGE_RATE)
                shares = capital / exec_price
            portfolio_lgbm.append(float(capital))
            portfolio_hold.append(float(shares * raw_price))

    if len(y_pred_all) < 2:
        print(f"⚠️ Skipping {ticker} — not enough data for evaluation.")
        return

    # === Metrics ===
    y_pred_all = np.array(y_pred_all[:len(y_true_all)])
    y_true_all = np.array(y_true_all[:len(y_pred_all)])
    accuracy = accuracy_score(y_true_all, y_pred_all)
    precision = precision_score(y_true_all, y_pred_all)
    recall = recall_score(y_true_all, y_pred_all)
    f1 = f1_score(y_true_all, y_pred_all)
    returns = np.diff(portfolio_lgbm) / (np.array(portfolio_lgbm[:-1]) + 1e-6)
    sharpe = np.mean(returns) / (np.std(returns) + 1e-6) * np.sqrt(252)
    drawdown = np.max(np.maximum.accumulate(portfolio_lgbm) - portfolio_lgbm)
    final_portfolio = portfolio_lgbm[-1]
    final_hold = portfolio_hold[-1]

    print(f"✅ {ticker} | Final: {final_portfolio:.2f} | Buy&Hold: {final_hold:.2f} | Sharpe: {sharpe:.3f} | Accuracy: {accuracy:.3f}")
    model.booster_.save_model(f"{SAVE_DIR}/model_{ticker}.txt")

    # === Save Results ===
    metrics_df = pd.DataFrame([{
        "Ticker": ticker,
        "Final_Portfolio": round(final_portfolio, 2),
        "Final_Hold": round(final_hold, 2),
        "Return_%": round((final_portfolio - initial_cash) / initial_cash * 100, 2),
        "Return_Hold_%": round((final_hold - initial_cash) / initial_cash * 100, 2),
        "Sharpe": round(sharpe, 4),
        "Accuracy": round(accuracy, 4),
        "Precision": round(precision, 4),
        "Recall": round(recall, 4),
        "F1_Score": round(f1, 4),
        "Drawdown": round(drawdown, 2)
    }])
    metrics_df.to_csv(f"{RESULTS_DIR}/metrics_{ticker}.csv", index=False)

    summary_path = os.path.join(RESULTS_DIR, "lightgbm_walkforward_summary.csv")
    if os.path.exists(summary_path):
        all_metrics = pd.read_csv(summary_path)
        all_metrics = all_metrics[all_metrics["Ticker"] != ticker]
        all_metrics = pd.concat([all_metrics, metrics_df], ignore_index=True)
    else:
        all_metrics = metrics_df
    all_metrics.to_csv(summary_path, index=False)

    # === Plotting ===
    plt.figure(figsize=(12, 6))
    plt.plot(portfolio_lgbm, label="LGBM Strategy")
    plt.plot(portfolio_hold, label="Buy & Hold")
    plt.title(f"{ticker} Portfolio Value")
    plt.xlabel("Step")
    plt.ylabel("Portfolio Value")
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"{RESULTS_DIR}/plots/{ticker}_portfolio_plot.png")
    plt.close()

    print(f"📁 Saved all artifacts for {ticker}")

# === Run All Tickers ===
for ticker in TICKERS:
    run_lgbm_walkforward(ticker)
    gc.collect()
    time.sleep(1)



📊 Running walkforward LGBM for AAPL


/tmp/ipython-input-21-543272446.py:68: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period="720d", interval="1h", progress=False)


✅ Saved scaler and features for AAPL
[LightGBM] [Warning] Unknown parameter: warm_start
[LightGBM] [Warning] Unknown parameter: warm_start
[LightGBM] [Info] Number of positive: 1286, number of negative: 1214
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000204 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2040
[LightGBM] [Info] Number of data points in the train set: 2500, number of used features: 8
[LightGBM] [Warning] Unknown parameter: warm_start
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.514400 -> initscore=0.057616
[LightGBM] [Info] Start training from score 0.057616
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[10]	valid_0's binary_logloss: 0.691907
[LightGBM] [Warning] Unknown parameter: warm_start
[LightGBM] [Warning] Unknown parameter: warm_start
[LightGBM] [Warning] Unknown parameter: warm_start
[LightGBM] [Warning] Unknown para

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/tmp/ipython-input-21-543272446.py:137: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  portfolio_lgbm.append(float(capital))
/tmp/ipython-input-21-543272446.py:138: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  portfolio_hold.append(float(shares * raw_price))
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and

[LightGBM] [Warning] Unknown parameter: warm_start
[LightGBM] [Warning] Unknown parameter: warm_start
[LightGBM] [Warning] Unknown parameter: warm_start
[LightGBM] [Info] Number of positive: 1308, number of negative: 1192
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000217 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2040
[LightGBM] [Info] Number of data points in the train set: 2500, number of used features: 8
[LightGBM] [Warning] Unknown parameter: warm_start
[LightGBM] [Warning] Unknown parameter: warm_start
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[12]	valid_0's binary_logloss: 0.695946
[LightGBM] [Warning] Unknown parameter: warm_start
[LightGBM] [Warning] Unknown parameter: warm_start
[LightGBM] [Warning] Unknown parameter: warm_start
[LightGBM] [Warning] Unknown parameter: warm_start
[LightGBM] [Info] Number of positive: 1296, numbe

In [23]:
#Noise Filtering
!pip install PyWavelets

In [46]:
# === Imports ===
import os
import gc
import json
import time
import random
import joblib
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

from datetime import datetime
from collections import deque

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans

from cuml.ensemble import RandomForestClassifier
import cupy as cp
import cudf
import cuml
import dask
import numba

import gymnasium as gym
import gym_anytrading
from gym_anytrading.envs import StocksEnv
from gymnasium.spaces import Box

from stable_baselines3 import PPO, SAC
from stable_baselines3.common.noise import NormalActionNoise
from stable_baselines3.common.vec_env import DummyVecEnv
import pywt

# === Ticker List and Config ===
ticker_list = [
    'AAPL', 'TSLA', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'META', 'BRK-B', 'JPM', 'JNJ',
    'XOM', 'V', 'PG', 'UNH', 'MA', 'HD', 'LLY', 'MRK', 'PEP', 'KO',
    'BAC', 'ABBV', 'AVGO', 'PFE', 'COST', 'CSCO', 'TMO', 'ABT', 'ACN', 'WMT',
    'MCD', 'ADBE', 'DHR', 'CRM', 'NKE', 'INTC', 'QCOM', 'NEE', 'AMD', 'TXN',
    'AMGN', 'UPS', 'LIN', 'PM', 'UNP', 'BMY', 'LOW', 'RTX', 'CVX', 'IBM',
    'GE', 'SBUX', 'ORCL'
]

strategy_name = "sac_ppo_td3_multi_stock_v1"

CONFIG = {
    'symbols': [],
    'period': '720d',
    'interval': '1h',
    'target': 'Target',
    'sharpe_threshold': 1.5,
    'return_threshold': 1.25,
    'strategy_name': strategy_name
}

features_list = [
    "SMA_50", "EMA_20", "RSI_voladj", "MACD_voladj",
    "Signal_Line", "ATR_voladj", "OBV", "CCI_voladj"
]
SLIPPAGE_RATE = 0.001

# === Noise Filtering Helpers ===
def wavelet_denoise_series(series, wavelet="db1", level=1):
    coeff = pywt.wavedec(series, wavelet, mode="symmetric")
    coeff[1:] = [np.zeros_like(c) for c in coeff[1:]]
    denoised = pywt.waverec(coeff, wavelet, mode="symmetric")
    return np.asarray(denoised).reshape(-1)

def apply_noise_filtering(df):
    for col in ["RSI", "MACD", "CCI", "OBV"]:
        if col in df.columns:
            denoised = wavelet_denoise_series(df[col].values)
            df[col + "_voladj"] = pd.Series(denoised[:len(df)], index=df.index)
    for col in ["ATR"]:
        if col in df.columns:
            denoised = wavelet_denoise_series(df[col].values)
            df[col + "_voladj"] = pd.Series(denoised[:len(df)], index=df.index)
    return df

# === Market Regime Detection ===
def detect_market_regimes(df, window=48, n_clusters=3):
    df = df.copy()
    returns = df['Close'].pct_change().rolling(window).sum().dropna()
    X = returns.values.reshape(-1, 1)
    kmeans = KMeans(n_clusters=n_clusters, random_state=42).fit(X)
    df['Regime'] = -1
    df.loc[returns.index, 'Regime'] = kmeans.labels_
    return df

# === Download Function ===
def download_stock_data(ticker, period="720d", interval="1h", max_retries=5):
    for attempt in range(1, max_retries + 1):
        try:
            print(f"Attempt {attempt}: Downloading {ticker}...")
            df = yf.download(ticker, period=period, interval=interval)
            if not df.empty:
                if isinstance(df.columns, pd.MultiIndex):
                    df.columns = df.columns.droplevel(1)
                df.reset_index(inplace=True)
                df['Symbol'] = ticker
                return df
            raise ValueError("Empty data")
        except Exception as e:
            print(f"Error: {e}. Retrying in {attempt * 5} sec...")
            time.sleep(attempt * 5)
    print(f"Failed to download {ticker}")
    return None

# === Feature Engineering ===
def compute_enhanced_features(df):
    df = df.copy()
    df = df.loc[:, ~df.columns.duplicated()]

    df['SMA_50'] = df['Close'].rolling(window=50).mean()
    df['EMA_20'] = df['Close'].ewm(span=20).mean()
    delta = df['Close'].diff()
    gain = delta.where(delta > 0, 0).rolling(window=14).mean()
    loss = -delta.where(delta < 0, 0).rolling(window=14).mean()
    rs = gain / (loss + 1e-6)
    df['RSI'] = 100 - (100 / (1 + rs))
    df['MACD'] = df['Close'].ewm(span=12).mean() - df['Close'].ewm(span=26).mean()
    df['Signal_Line'] = df['MACD'].ewm(span=9).mean()
    df['ATR'] = df['High'].rolling(window=14).max() - df['Low'].rolling(window=14).min()
    df['OBV'] = (np.sign(df['Close'].diff()) * df['Volume']).fillna(0).cumsum()
    tp = (df['High'] + df['Low'] + df['Close']) / 3
    df['CCI'] = (tp - tp.rolling(20).mean()) / (0.015 * tp.rolling(20).std())

    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.dropna(inplace=True)
    return df

# === Main Pipeline ===
all_dfs = []

for ticker in ticker_list:
    df_single = download_stock_data(ticker, period=CONFIG['period'], interval=CONFIG['interval'])
    if df_single is not None:
        try:
            df_features = compute_enhanced_features(df_single)
            df_features = apply_noise_filtering(df_features)
            df_features = detect_market_regimes(df_features)
            all_dfs.append(df_features)
        except Exception as e:
            print(f"Feature engineering failed for {ticker}: {e}")
    else:
        print(f"Failed to download {ticker}")

if all_dfs:
    df = pd.concat(all_dfs, ignore_index=True)
    print(f"Combined dataset created with shape: {df.shape}")
else:
    df = pd.DataFrame()
    print("No data available.")

if not df.empty:
    df.to_csv("multi_stock_feature_engineered_dataset.csv", index=False)
    print("Saved locally to multi_stock_feature_engineered_dataset.csv")

    drive_path = "/content/drive/MyDrive/trading_data/"
    os.makedirs(drive_path, exist_ok=True)
    df.to_csv(os.path.join(drive_path, "multi_stock_feature_engineered_dataset.csv"), index=False)
    print(f"Also saved to Google Drive at {drive_path}multi_stock_feature_engineered_dataset.csv")


Attempt 1: Downloading AAPL...


/tmp/ipython-input-46-3277607469.py:100: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period=period, interval=interval)
[*********************100%***********************]  1 of 1 completed


Attempt 1: Downloading TSLA...


/tmp/ipython-input-46-3277607469.py:100: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period=period, interval=interval)
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-46-3277607469.py:100: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period=period, interval=interval)


Attempt 1: Downloading MSFT...


...

Attempt 1: Downloading SBUX...


[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-46-3277607469.py:100: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period=period, interval=interval)


Attempt 1: Downloading ORCL...


[*********************100%***********************]  1 of 1 completed


Combined dataset created with shape: (263250, 21)
Saved locally to multi_stock_feature_engineered_dataset.csv
Also saved to Google Drive at /content/drive/MyDrive/trading_data/multi_stock_feature_engineered_dataset.csv


In [49]:
#Market Regime Detection

# === Imports ===
import os
import gc
import json
import time
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import pywt
from datetime import datetime
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from lightgbm import LGBMClassifier, early_stopping, log_evaluation

# === Configuration ===
SAVE_DIR = "/content/drive/MyDrive/QuantConnect/results_lightgbm/lightgbm_walkforward_models"
RESULTS_DIR = "/content/drive/MyDrive/QuantConnect/results_lightgbm"
os.makedirs(SAVE_DIR, exist_ok=True)
os.makedirs(os.path.join(RESULTS_DIR, "plots"), exist_ok=True)
os.makedirs(os.path.join(RESULTS_DIR, "data"), exist_ok=True)

TICKERS = ['AAPL']
SLIPPAGE_RATE = 0.001
FEATURES = ["SMA_50", "EMA_20", "RSI_voladj", "MACD_voladj", "Signal_Line", "ATR_voladj", "OBV", "CCI_voladj"]

# === Wavelet Denoising ===
def wavelet_denoise_series(series, wavelet="db1"):
    coeff = pywt.wavedec(series, wavelet, mode="symmetric")
    coeff[1:] = [np.zeros_like(c) for c in coeff[1:]]
    return pywt.waverec(coeff, wavelet, mode="symmetric")[:len(series)]

def apply_noise_filtering(df):
    for col in ["RSI", "MACD", "CCI", "OBV", "ATR"]:
        if col in df.columns:
            df[f"{col}_voladj"] = wavelet_denoise_series(df[col].values)
    return df

# === Market Regime Detection ===
def detect_market_regimes(df, window=100):
    returns = df['Close'].pct_change().dropna()
    vol = df['Close'].rolling(window).std().dropna()
    regime_df = pd.DataFrame({"returns": returns, "volatility": vol})
    regime_df.dropna(inplace=True)
    kmeans = KMeans(n_clusters=3, random_state=42).fit(regime_df)
    regimes = pd.Series(index=regime_df.index, data=kmeans.labels_)
    df["Regime"] = regimes.reindex(df.index).ffill().bfill()
    return df

# === Feature Engineering ===
def compute_features(df):
    df['SMA_50'] = df['Close'].rolling(window=50).mean()
    df['EMA_20'] = df['Close'].ewm(span=20).mean()
    delta = df['Close'].diff()
    gain = delta.where(delta > 0, 0).rolling(14).mean()
    loss = -delta.where(delta < 0, 0).rolling(14).mean()
    rs = gain / (loss + 1e-6)
    df['RSI'] = 100 - (100 / (1 + rs))
    df['MACD'] = df['Close'].ewm(span=12).mean() - df['Close'].ewm(span=26).mean()
    df['Signal_Line'] = df['MACD'].ewm(span=9).mean()
    df['ATR'] = df['High'].rolling(14).max() - df['Low'].rolling(14).min()
    df['OBV'] = (np.sign(df['Close'].diff()) * df['Volume']).fillna(0).cumsum()
    tp = (df['High'] + df['Low'] + df['Close']) / 3
    df['CCI'] = (tp - tp.rolling(20).mean()) / (0.015 * tp.rolling(20).std())
    df.dropna(inplace=True)
    df = apply_noise_filtering(df)
    df = detect_market_regimes(df)
    df['Target'] = (df['Close'].shift(-1) > df['Close']).astype(int)
    return df.dropna()

# === LGBM Walkforward Trainer ===
def run_lgbm_walkforward(ticker, window_size=3000, step_size=500, initial_cash=100000):
    print(f"\n📊 Running walkforward LGBM for {ticker}")
    df = yf.download(ticker, period="720d", interval="1h", progress=False)

    # ✅ Flatten columns if MultiIndex
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.droplevel(1)

    if df.empty:
        print("⚠️ Empty dataframe. Skipping.")
        return

    df = compute_enhanced_features(df)
    df = apply_noise_filtering(df)
    df = detect_market_regimes(df)  # optional, if implemented

    df["Target"] = (df["Close"].shift(-1) > df["Close"]).astype(int)
    df.dropna(inplace=True)

    # 🔍 DEBUG: Check all columns after feature engineering
    print("Columns in DataFrame after feature engineering:", df.columns.tolist())

    if df.empty or not all(col in df.columns for col in features_list):
        print("⚠️ Missing required columns after feature engineering.")
        return

    scaler = MinMaxScaler()
    portfolio_lgbm, portfolio_hold, y_pred_all, y_true_all = [], [], [], []
    capital = initial_cash
    shares = initial_cash / df['Close'].iloc[0]

    model = LGBMClassifier(n_estimators=100, learning_rate=0.05, warm_start=True, random_state=42)
    model_initialized = False

    for start in range(0, len(df) - window_size, step_size):
        segment = df.iloc[start:start+window_size].copy()
        if segment.shape[0] < window_size:
            print(f"⚠️ Not enough data at step {start}. Skipping.")
            continue

        X = segment[features_list]
        y = segment['Target']
        X_scaled = scaler.fit_transform(X)
        X_train, X_test = X_scaled[:-step_size], X_scaled[-step_size:]
        y_train, y_test = y[:-step_size], y[-step_size:]

        print(f"🔁 Window {start} — X_train: {X_train.shape}, X_test: {X_test.shape}")

        if X_test.shape[0] == 0 or len(y_test) == 0:
            print("⚠️ Skipping: No test data available.")
            continue

        if not model_initialized:
            model.fit(X_train, y_train,
                      eval_set=[(X_test, y_test)],
                      callbacks=[early_stopping(20), log_evaluation(0)])
            model_initialized = True
        else:
            model.n_estimators += 100
            model.fit(X_train, y_train,
                      eval_set=[(X_test, y_test)],
                      callbacks=[early_stopping(20), log_evaluation(0)],
                      init_model=model)

        preds = model.predict(X_test)
        if preds is None or len(preds) == 0:
            print("⚠️ No predictions made. Skipping step.")
            continue

        preds = (preds > 0.5).astype(int)
        prices = segment['Close'].values[-step_size:]

        if len(preds) != step_size or len(prices) != step_size:
            print("⚠️ Mismatch in prediction and price length.")
            continue

        y_pred_all.extend(preds)
        y_true_all.extend(y_test.tolist())

        acc_window = accuracy_score(y_test, preds)
        if acc_window < 0.5:
            print("⚠️ Accuracy dropped. Resetting model.")
            model = LGBMClassifier(n_estimators=100, learning_rate=0.05, random_state=42)
            model_initialized = False

        for i, pred in enumerate(preds):
            raw_price = prices[i]
            if pred:
                exec_price = raw_price * (1 + SLIPPAGE_RATE)
                capital = shares * exec_price
            else:
                exec_price = raw_price * (1 - SLIPPAGE_RATE)
                shares = capital / exec_price
            portfolio_lgbm.append(float(capital))
            portfolio_hold.append(float(shares * raw_price))

    if len(y_pred_all) < 2:
        print(f"⚠️ Skipping {ticker} — not enough data for evaluation.")
        return

    y_pred_all = np.array(y_pred_all[:len(y_true_all)])
    y_true_all = np.array(y_true_all[:len(y_pred_all)])
    accuracy = accuracy_score(y_true_all, y_pred_all)
    precision = precision_score(y_true_all, y_pred_all)
    recall = recall_score(y_true_all, y_pred_all)
    f1 = f1_score(y_true_all, y_pred_all)
    returns = np.diff(portfolio_lgbm) / (np.array(portfolio_lgbm[:-1]) + 1e-6)
    sharpe = np.mean(returns) / (np.std(returns) + 1e-6) * np.sqrt(252)
    drawdown = np.max(np.maximum.accumulate(portfolio_lgbm) - portfolio_lgbm)
    final_portfolio = portfolio_lgbm[-1]
    final_hold = portfolio_hold[-1]

    print(f"✅ {ticker} | Final: {final_portfolio:.2f} | Buy&Hold: {final_hold:.2f} | Sharpe: {sharpe:.3f} | Accuracy: {accuracy:.3f}")
    model.booster_.save_model(f"{SAVE_DIR}/model_{ticker}.txt")

    metrics_df = pd.DataFrame([{
        "Ticker": ticker,
        "Final_Portfolio": round(final_portfolio, 2),
        "Final_Hold": round(final_hold, 2),
        "Return_%": round((final_portfolio - initial_cash) / initial_cash * 100, 2),
        "Return_Hold_%": round((final_hold - initial_cash) / initial_cash * 100, 2),
        "Sharpe": round(sharpe, 4),
        "Accuracy": round(accuracy, 4),
        "Precision": round(precision, 4),
        "Recall": round(recall, 4),
        "F1_Score": round(f1, 4),
        "Drawdown": round(drawdown, 2)
    }])
    metrics_df.to_csv(f"{RESULTS_DIR}/metrics_{ticker}.csv", index=False)

    summary_path = os.path.join(RESULTS_DIR, "lightgbm_walkforward_summary.csv")
    if os.path.exists(summary_path):
        all_metrics = pd.read_csv(summary_path)
        all_metrics = all_metrics[all_metrics["Ticker"] != ticker]
        all_metrics = pd.concat([all_metrics, metrics_df], ignore_index=True)
    else:
        all_metrics = metrics_df
    all_metrics.to_csv(summary_path, index=False)

    plt.figure(figsize=(12, 6))
    plt.plot(portfolio_lgbm, label="LGBM Strategy")
    plt.plot(portfolio_hold, label="Buy & Hold")
    plt.title(f"{ticker} Portfolio Value")
    plt.xlabel("Step")
    plt.ylabel("Portfolio Value")
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"{RESULTS_DIR}/plots/{ticker}_portfolio_plot.png")
    plt.close()

    print(f"📁 Saved all artifacts for {ticker}")
    # === Run All Tickers ===
for ticker in TICKERS:
    run_lgbm_walkforward(ticker)
    gc.collect()
    time.sleep(1)



📊 Running walkforward LGBM for AAPL


/tmp/ipython-input-49-995312704.py:76: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period="720d", interval="1h", progress=False)


Columns in DataFrame after feature engineering: ['Close', 'High', 'Low', 'Open', 'Volume', 'SMA_50', 'EMA_20', 'RSI', 'MACD', 'Signal_Line', 'ATR', 'OBV', 'CCI', 'RSI_voladj', 'MACD_voladj', 'CCI_voladj', 'OBV_voladj', 'ATR_voladj', 'Regime', 'Target']
🔁 Window 0 — X_train: (2500, 8), X_test: (500, 8)
[LightGBM] [Warning] Unknown parameter: warm_start
[LightGBM] [Warning] Unknown parameter: warm_start
[LightGBM] [Info] Number of positive: 1286, number of negative: 1214
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000128 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1020
[LightGBM] [Info] Number of data points in the train set: 2500, number of used features: 4
[LightGBM] [Warning] Unknown parameter: warm_start
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.514400 -> initscore=0.057616
[LightGBM] [Info] Start training from score 0.057616
Training until validation scores don't improve for 20 ro

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11

📁 Saved all artifacts for AAPL


In [51]:
#Latency & Broker Integration
# === Imports ===
import os
import time
import numpy as np
import pandas as pd
import yfinance as yf
import pywt
from sklearn.cluster import KMeans

# === Configuration ===
TICKERS = [
    'AAPL', 'TSLA', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'META', 'BRK-B', 'JPM', 'JNJ',
    'XOM', 'V', 'PG', 'UNH', 'MA', 'HD', 'LLY', 'MRK', 'PEP', 'KO',
    'BAC', 'ABBV', 'AVGO', 'PFE', 'COST', 'CSCO', 'TMO', 'ABT', 'ACN', 'WMT',
    'MCD', 'ADBE', 'DHR', 'CRM', 'NKE', 'INTC', 'QCOM', 'NEE', 'AMD', 'TXN',
    'AMGN', 'UPS', 'LIN', 'PM', 'UNP', 'BMY', 'LOW', 'RTX', 'CVX', 'IBM',
    'GE', 'SBUX', 'ORCL'
]
PERIOD = "720d"
INTERVAL = "1h"
OUTPUT_PATH = "/content/drive/MyDrive/trading_data/multi_stock_feature_engineered_dataset.csv"

# === Wavelet Denoising Function ===
def wavelet_denoise_series(series, wavelet="db1"):
    coeff = pywt.wavedec(series, wavelet, mode="symmetric")
    coeff[1:] = [np.zeros_like(c) for c in coeff[1:]]
    return pywt.waverec(coeff, wavelet, mode="symmetric")[:len(series)]

# === Apply Noise Filtering ===
def apply_noise_filtering(df):
    for col in ["RSI", "MACD", "CCI", "OBV", "ATR"]:
        if col in df.columns:
            df[f"{col}_voladj"] = wavelet_denoise_series(df[col].values)
    return df

# === Market Regime Detection ===
def detect_market_regimes(df, window=100):
    returns = df['Close'].pct_change().dropna()
    vol = df['Close'].rolling(window).std().dropna()
    regime_df = pd.DataFrame({"returns": returns, "volatility": vol}).dropna()
    if regime_df.empty or len(regime_df) < 50:
        df["Regime"] = 1  # default regime
        return df
    kmeans = KMeans(n_clusters=3, random_state=42).fit(regime_df)
    regimes = pd.Series(index=regime_df.index, data=kmeans.labels_)
    df["Regime"] = regimes.reindex(df.index).ffill().bfill()
    return df

# === Feature Engineering ===
def compute_enhanced_features(df):
    df = df.copy()
    df = df.loc[:, ~df.columns.duplicated()]
    df['SMA_50'] = df['Close'].rolling(window=50).mean()
    df['EMA_20'] = df['Close'].ewm(span=20).mean()
    delta = df['Close'].diff()
    gain = delta.where(delta > 0, 0).rolling(14).mean()
    loss = -delta.where(delta < 0, 0).rolling(14).mean()
    rs = gain / (loss + 1e-6)
    df['RSI'] = 100 - (100 / (1 + rs))
    df['MACD'] = df['Close'].ewm(span=12).mean() - df['Close'].ewm(span=26).mean()
    df['Signal_Line'] = df['MACD'].ewm(span=9).mean()
    df['ATR'] = df['High'].rolling(14).max() - df['Low'].rolling(14).min()
    df['OBV'] = (np.sign(df['Close'].diff()) * df['Volume']).fillna(0).cumsum()
    tp = (df['High'] + df['Low'] + df['Close']) / 3
    df['CCI'] = (tp - tp.rolling(20).mean()) / (0.015 * tp.rolling(20).std())
    df = apply_noise_filtering(df)
    df = detect_market_regimes(df)
    df['Target'] = (df['Close'].shift(-1) > df['Close']).astype(int)
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.dropna(inplace=True)
    return df

# === Download Function ===
def download_stock_data(ticker, period="720d", interval="1h", max_retries=3):
    for attempt in range(1, max_retries + 1):
        try:
            print(f"📥 Attempt {attempt}: Downloading {ticker}...")
            df = yf.download(ticker, period=period, interval=interval)
            if not df.empty:
                if isinstance(df.columns, pd.MultiIndex):
                    df.columns = df.columns.droplevel(1)
                df.reset_index(inplace=True)
                df['Symbol'] = ticker
                return df
            raise ValueError("Empty data")
        except Exception as e:
            print(f"Error: {e}. Retrying in {attempt * 5}s...")
            time.sleep(attempt * 5)
    print(f"❌ Failed to download {ticker}")
    return None

# === Pipeline ===
all_dfs = []

for ticker in TICKERS:
    raw_df = download_stock_data(ticker, period=PERIOD, interval=INTERVAL)
    if raw_df is not None:
        try:
            feat_df = compute_enhanced_features(raw_df)
            all_dfs.append(feat_df)
        except Exception as e:
            print(f"⚠️ Feature engineering failed for {ticker}: {e}")
    else:
        print(f"⚠️ Skipping {ticker} due to download failure.")

# === Save Output ===
if all_dfs:
    final_df = pd.concat(all_dfs, ignore_index=True)
    print(f"✅ Combined dataset shape: {final_df.shape}")
    final_df.to_csv("multi_stock_feature_engineered_dataset.csv", index=False)
    os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)
    final_df.to_csv(OUTPUT_PATH, index=False)
    print(f"📁 Saved to: {OUTPUT_PATH}")
else:
    print("❌ No data collected.")


📥 Attempt 1: Downloading AAPL...


/tmp/ipython-input-51-4194488514.py:79: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period=period, interval=interval)
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-51-4194488514.py:79: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period=period, interval=interval)


📥 Attempt 1: Downloading TSLA...


[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-51-4194488514.py:79: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period=period, interval=interval)


📥 Attempt 1: Downloading MSFT...


...

📥 Attempt 1: Downloading SBUX...


[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-51-4194488514.py:79: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period=period, interval=interval)


📥 Attempt 1: Downloading ORCL...


[*********************100%***********************]  1 of 1 completed


✅ Combined dataset shape: (48759, 22)
📁 Saved to: /content/drive/MyDrive/trading_data/multi_stock_feature_engineered_dataset.csv


In [53]:
#Market Regime Detection

# === Imports ===
import os
import gc
import json
import time
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import pywt
from datetime import datetime
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from lightgbm import LGBMClassifier, early_stopping, log_evaluation

# === Configuration ===
SAVE_DIR = "/content/drive/MyDrive/QuantConnect/results_lightgbm/lightgbm_walkforward_models"
RESULTS_DIR = "/content/drive/MyDrive/QuantConnect/results_lightgbm"
os.makedirs(SAVE_DIR, exist_ok=True)
os.makedirs(os.path.join(RESULTS_DIR, "plots"), exist_ok=True)
os.makedirs(os.path.join(RESULTS_DIR, "data"), exist_ok=True)

TICKERS = ['AAPL']
SLIPPAGE_RATE = 0.001
FEATURES = ["SMA_50", "EMA_20", "RSI_voladj", "MACD_voladj", "Signal_Line", "ATR_voladj", "OBV", "CCI_voladj"]

# === Wavelet Denoising ===
def wavelet_denoise_series(series, wavelet="db1"):
    coeff = pywt.wavedec(series, wavelet, mode="symmetric")
    coeff[1:] = [np.zeros_like(c) for c in coeff[1:]]
    return pywt.waverec(coeff, wavelet, mode="symmetric")[:len(series)]

def apply_noise_filtering(df):
    for col in ["RSI", "MACD", "CCI", "OBV", "ATR"]:
        if col in df.columns:
            df[f"{col}_voladj"] = wavelet_denoise_series(df[col].values)
    return df

# === Market Regime Detection ===
def detect_market_regimes(df, window=100):
    returns = df['Close'].pct_change().dropna()
    vol = df['Close'].rolling(window).std().dropna()
    regime_df = pd.DataFrame({"returns": returns, "volatility": vol})
    regime_df.dropna(inplace=True)
    kmeans = KMeans(n_clusters=3, random_state=42).fit(regime_df)
    regimes = pd.Series(index=regime_df.index, data=kmeans.labels_)
    df["Regime"] = regimes.reindex(df.index).ffill().bfill()
    return df

# === Feature Engineering ===
def compute_features(df):
    df['SMA_50'] = df['Close'].rolling(window=50).mean()
    df['EMA_20'] = df['Close'].ewm(span=20).mean()
    delta = df['Close'].diff()
    gain = delta.where(delta > 0, 0).rolling(14).mean()
    loss = -delta.where(delta < 0, 0).rolling(14).mean()
    rs = gain / (loss + 1e-6)
    df['RSI'] = 100 - (100 / (1 + rs))
    df['MACD'] = df['Close'].ewm(span=12).mean() - df['Close'].ewm(span=26).mean()
    df['Signal_Line'] = df['MACD'].ewm(span=9).mean()
    df['ATR'] = df['High'].rolling(14).max() - df['Low'].rolling(14).min()
    df['OBV'] = (np.sign(df['Close'].diff()) * df['Volume']).fillna(0).cumsum()
    tp = (df['High'] + df['Low'] + df['Close']) / 3
    df['CCI'] = (tp - tp.rolling(20).mean()) / (0.015 * tp.rolling(20).std())
    df.dropna(inplace=True)
    df = apply_noise_filtering(df)
    df = detect_market_regimes(df)
    df['Target'] = (df['Close'].shift(-1) > df['Close']).astype(int)
    return df.dropna()

# === LGBM Walkforward Trainer ===
def run_lgbm_walkforward(ticker, window_size=2000, step_size=250, initial_cash=100000):
    print(f"\n📊 Running walkforward LGBM for {ticker}")
    df = yf.download(ticker, period="720d", interval="1h", progress=False)

    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.droplevel(1)

    if df.empty:
        print("⚠️ Empty dataframe. Skipping.")
        return

    df = compute_features(df)

    if df.empty or not all(col in df.columns for col in FEATURES):
        print("⚠️ Missing required columns after feature engineering.")
        return

    scaler = MinMaxScaler()
    portfolio_lgbm, portfolio_hold, y_pred_all, y_true_all = [], [], [], []
    capital = initial_cash
    shares = initial_cash / df['Close'].iloc[0]

    model = LGBMClassifier(n_estimators=100, learning_rate=0.05, random_state=42)
    model_initialized = False

    for start in range(0, len(df) - window_size, step_size):
        segment = df.iloc[start:start+window_size].copy()
        if segment.shape[0] < window_size:
            print(f"⚠️ Not enough data at step {start}. Skipping.")
            continue

        X = segment[FEATURES]
        y = segment['Target']
        X_scaled = scaler.fit_transform(X)
        X_train, X_test = X_scaled[:-step_size], X_scaled[-step_size:]
        y_train, y_test = y[:-step_size], y[-step_size:]

        print(f"🔁 Window {start} — X_train: {X_train.shape}, X_test: {X_test.shape}")

        if len(y_test) == 0:
            print("⚠️ Skipping: No test data available.")
            continue

        if not model_initialized:
            model.fit(X_train, y_train,
                      eval_set=[(X_test, y_test)],
                      callbacks=[early_stopping(20), log_evaluation(0)])
            model_initialized = True
        else:
            model.n_estimators += 100
            model.fit(X_train, y_train,
                      eval_set=[(X_test, y_test)],
                      callbacks=[early_stopping(20), log_evaluation(0)],
                      init_model=model)

        preds = model.predict(X_test)
        if preds is None or len(preds) == 0:
            print("⚠️ No predictions made. Skipping step.")
            continue

        preds = (preds > 0.5).astype(int)
        prices = segment['Close'].values[-step_size:]

        if len(preds) != len(prices):
            print("⚠️ Prediction/price length mismatch. Skipping.")
            continue

        y_pred_all.extend(preds)
        y_true_all.extend(y_test.tolist())

        acc_window = accuracy_score(y_test, preds)
        if acc_window < 0.5:
            print("⚠️ Accuracy dropped. Resetting model.")
            model = LGBMClassifier(n_estimators=100, learning_rate=0.05, random_state=42)
            model_initialized = False

        for i, pred in enumerate(preds):
            raw_price = prices[i]
            if pred:
                exec_price = raw_price * (1 + SLIPPAGE_RATE)
                capital = shares * exec_price
            else:
                exec_price = raw_price * (1 - SLIPPAGE_RATE)
                shares = capital / exec_price
            portfolio_lgbm.append(float(capital))
            portfolio_hold.append(float(shares * raw_price))

    print("🔎 Predictions collected:", len(y_pred_all), "Truths:", len(y_true_all))
    if len(y_pred_all) < 2:
        print(f"⚠️ Skipping {ticker} — not enough data for evaluation.")
        return

    y_pred_all = np.array(y_pred_all[:len(y_true_all)])
    y_true_all = np.array(y_true_all[:len(y_pred_all)])
    accuracy = accuracy_score(y_true_all, y_pred_all)
    precision = precision_score(y_true_all, y_pred_all)
    recall = recall_score(y_true_all, y_pred_all)
    f1 = f1_score(y_true_all, y_pred_all)
    returns = np.diff(portfolio_lgbm) / (np.array(portfolio_lgbm[:-1]) + 1e-6)
    sharpe = np.mean(returns) / (np.std(returns) + 1e-6) * np.sqrt(252)
    drawdown = np.max(np.maximum.accumulate(portfolio_lgbm) - portfolio_lgbm)
    final_portfolio = portfolio_lgbm[-1]
    final_hold = portfolio_hold[-1]

    print(f"✅ {ticker} | Final: {final_portfolio:.2f} | Buy&Hold: {final_hold:.2f} | Sharpe: {sharpe:.3f} | Accuracy: {accuracy:.3f}")
    model.booster_.save_model(f"{SAVE_DIR}/model_{ticker}.txt")

    metrics_df = pd.DataFrame([{
        "Ticker": ticker,
        "Final_Portfolio": round(final_portfolio, 2),
        "Final_Hold": round(final_hold, 2),
        "Return_%": round((final_portfolio - initial_cash) / initial_cash * 100, 2),
        "Return_Hold_%": round((final_hold - initial_cash) / initial_cash * 100, 2),
        "Sharpe": round(sharpe, 4),
        "Accuracy": round(accuracy, 4),
        "Precision": round(precision, 4),
        "Recall": round(recall, 4),
        "F1_Score": round(f1, 4),
        "Drawdown": round(drawdown, 2)
    }])
    metrics_df.to_csv(f"{RESULTS_DIR}/metrics_{ticker}.csv", index=False)

    summary_path = os.path.join(RESULTS_DIR, "lightgbm_walkforward_summary.csv")
    if os.path.exists(summary_path):
        all_metrics = pd.read_csv(summary_path)
        all_metrics = all_metrics[all_metrics["Ticker"] != ticker]
        all_metrics = pd.concat([all_metrics, metrics_df], ignore_index=True)
    else:
        all_metrics = metrics_df
    all_metrics.to_csv(summary_path, index=False)

    plt.figure(figsize=(12, 6))
    plt.plot(portfolio_lgbm, label="LGBM Strategy")
    plt.plot(portfolio_hold, label="Buy & Hold")
    plt.title(f"{ticker} Portfolio Value")
    plt.xlabel("Step")
    plt.ylabel("Portfolio Value")
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"{RESULTS_DIR}/plots/{ticker}_portfolio_plot.png")
    plt.close()

    print(f"📁 Saved all artifacts for {ticker}")
    # === Run All Tickers ===
for ticker in TICKERS:
    run_lgbm_walkforward(ticker)
    gc.collect()
    time.sleep(1)



📊 Running walkforward LGBM for AAPL


/tmp/ipython-input-53-3665151829.py:78: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period="720d", interval="1h", progress=False)


🔁 Window 0 — X_train: (1750, 8), X_test: (250, 8)
[LightGBM] [Info] Number of positive: 879, number of negative: 871
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001221 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1020
[LightGBM] [Info] Number of data points in the train set: 1750, number of used features: 4
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.502286 -> initscore=0.009143
[LightGBM] [Info] Start training from score 0.009143
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[2]	valid_0's binary_logloss: 0.692109
🔁 Window 250 — X_train: (1750, 8), X_test: (250, 8)
[LightGBM] [Info] Number of positive: 897, number of negative: 853
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000158 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1020
[LightGBM] [In

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11

[LightGBM] [Info] Number of positive: 910, number of negative: 840
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000139 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1020
[LightGBM] [Info] Number of data points in the train set: 1750, number of used features: 4
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[12]	valid_0's binary_logloss: 0.700486
🔁 Window 750 — X_train: (1750, 8), X_test: (250, 8)
[LightGBM] [Info] Number of positive: 928, number of negative: 822
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000158 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1020
[LightGBM] [Info] Number of data points in the train set: 1750, number of used features: 4
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[14]	val

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11

Early stopping, best iteration is:
[10]	valid_0's binary_logloss: 0.695588
🔎 Predictions collected: 3000 Truths: 3000
✅ AAPL | Final: 138880.09 | Buy&Hold: 138741.35 | Sharpe: 0.462 | Accuracy: 0.525


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


📁 Saved all artifacts for AAPL


In [58]:
# === Imports ===
import os
import gc
import json
import time
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import pywt
from datetime import datetime
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from lightgbm import LGBMClassifier, early_stopping, log_evaluation
from lightgbm.basic import LightGBMError

# === Configuration ===
SAVE_DIR = "/content/drive/MyDrive/QuantConnect/results_lightgbm/lightgbm_walkforward_models"
RESULTS_DIR = "/content/drive/MyDrive/QuantConnect/results_lightgbm"
os.makedirs(SAVE_DIR, exist_ok=True)
os.makedirs(os.path.join(RESULTS_DIR, "plots"), exist_ok=True)
os.makedirs(os.path.join(RESULTS_DIR, "data"), exist_ok=True)

TICKERS = [
    'AAPL', 'TSLA', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'META', 'BRK-B', 'JPM', 'JNJ',
    'XOM', 'V', 'PG', 'UNH', 'MA', 'HD', 'LLY', 'MRK', 'PEP', 'KO',
    'BAC', 'ABBV', 'AVGO', 'PFE', 'COST', 'CSCO', 'TMO', 'ABT', 'ACN', 'WMT',
    'MCD', 'ADBE', 'DHR', 'CRM', 'NKE', 'INTC', 'QCOM', 'NEE', 'AMD', 'TXN',
    'AMGN', 'UPS', 'LIN', 'PM', 'UNP', 'BMY', 'LOW', 'RTX', 'CVX', 'IBM',
    'GE', 'SBUX', 'ORCL'
]
SLIPPAGE_RATE = 0.001
FEATURES = ["SMA_50", "EMA_20", "RSI_voladj", "MACD_voladj", "Signal_Line", "ATR_voladj", "OBV", "CCI_voladj"]
MAX_EXPOSURE_PERCENT = 0.25  # 25% capital exposure cap
STOP_LOSS_THRESHOLD = 0.02    # 2% trailing stop-loss

# === Wavelet Denoising ===
def wavelet_denoise_series(series, wavelet="db1"):
    coeff = pywt.wavedec(series, wavelet, mode="symmetric")
    coeff[1:] = [np.zeros_like(c) for c in coeff[1:]]
    return pywt.waverec(coeff, wavelet, mode="symmetric")[:len(series)]

def apply_noise_filtering(df):
    for col in ["RSI", "MACD", "CCI", "OBV", "ATR"]:
        if col in df.columns:
            df[f"{col}_voladj"] = wavelet_denoise_series(df[col].values)
    return df

# === Market Regime Detection ===
def detect_market_regimes(df, window=100):
    returns = df['Close'].pct_change().dropna()
    vol = df['Close'].rolling(window).std().dropna()
    regime_df = pd.DataFrame({"returns": returns, "volatility": vol})
    regime_df.dropna(inplace=True)
    kmeans = KMeans(n_clusters=3, random_state=42).fit(regime_df)
    regimes = pd.Series(index=regime_df.index, data=kmeans.labels_)
    df["Regime"] = regimes.reindex(df.index).ffill().bfill()
    return df

# === Feature Engineering ===
def compute_features(df):
    df['SMA_50'] = df['Close'].rolling(window=50).mean()
    df['EMA_20'] = df['Close'].ewm(span=20).mean()
    delta = df['Close'].diff()
    gain = delta.where(delta > 0, 0).rolling(14).mean()
    loss = -delta.where(delta < 0, 0).rolling(14).mean()
    rs = gain / (loss + 1e-6)
    df['RSI'] = 100 - (100 / (1 + rs))
    df['MACD'] = df['Close'].ewm(span=12).mean() - df['Close'].ewm(span=26).mean()
    df['Signal_Line'] = df['MACD'].ewm(span=9).mean()
    df['ATR'] = df['High'].rolling(14).max() - df['Low'].rolling(14).min()
    df['OBV'] = (np.sign(df['Close'].diff()) * df['Volume']).fillna(0).cumsum()
    tp = (df['High'] + df['Low'] + df['Close']) / 3
    df['CCI'] = (tp - tp.rolling(20).mean()) / (0.015 * tp.rolling(20).std())
    df.dropna(inplace=True)
    df = apply_noise_filtering(df)
    df = detect_market_regimes(df)
    df['Target'] = (df['Close'].shift(-1) > df['Close']).astype(int)
    return df.dropna()

# === Risk Management Utilities ===
def get_max_position(capital):
    return capital * MAX_EXPOSURE_PERCENT

def apply_stop_loss(entry_price, current_price):
    return current_price < entry_price * (1 - STOP_LOSS_THRESHOLD)

# === Run All Tickers ===
failed_tickers = []

for ticker in TICKERS:
    try:
        run_lgbm_walkforward(ticker)
    except (LightGBMError, ValueError, KeyError) as e:
        print(f"❌ Failed to run for {ticker}: {e}")
        failed_tickers.append(ticker)
    gc.collect()
    time.sleep(1)

if failed_tickers:
    print(f"🚫 The following tickers failed: {failed_tickers}")



📊 Running walkforward LGBM for AAPL


/tmp/ipython-input-53-3665151829.py:78: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period="720d", interval="1h", progress=False)


🔁 Window 0 — X_train: (1750, 8), X_test: (250, 8)
[LightGBM] [Info] Number of positive: 879, number of negative: 871
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000139 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1020
[LightGBM] [Info] Number of data points in the train set: 1750, number of used features: 4
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.502286 -> initscore=0.009143
[LightGBM] [Info] Start training from score 0.009143
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[2]	valid_0's binary_logloss: 0.692109
🔁 Window 250 — X_train: (1750, 8), X_test: (250, 8)
[LightGBM] [Info] Number of positive: 897, number of negative: 853
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000144 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1020
[LightGBM] [In

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11

[LightGBM] [Info] Number of positive: 921, number of negative: 829
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000135 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1020
[LightGBM] [Info] Number of data points in the train set: 1750, number of used features: 4
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[25]	valid_0's binary_logloss: 0.682696
🔁 Window 1250 — X_train: (1750, 8), X_test: (250, 8)
[LightGBM] [Info] Number of positive: 902, number of negative: 848
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000155 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1020
[LightGBM] [Info] Number of data points in the train set: 1750, number of used features: 4
Training until validation scores don't improve for 20 rounds


...

🔁 Window 0 — X_train: (1750, 8), X_test: (250, 8)
[LightGBM] [Info] Number of positive: 890, number of negative: 860
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000097 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1020
[LightGBM] [Info] Number of data points in the train set: 1750, number of used features: 4
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.508571 -> initscore=0.034289
[LightGBM] [Info] Start training from score 0.034289
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[1]	valid_0's binary_logloss: 0.692724
🔁 Window 250 — X_train: (1750, 8), X_test: (250, 8)
[LightGBM] [Info] Number of positive: 914, number of negative: 836
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000081 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1020
[LightGBM] [In

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11

Early stopping, best iteration is:
[8]	valid_0's binary_logloss: 0.694927
⚠️ Accuracy dropped. Resetting model.
🔁 Window 2250 — X_train: (1750, 8), X_test: (250, 8)
[LightGBM] [Info] Number of positive: 908, number of negative: 842
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000117 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1020
[LightGBM] [Info] Number of data points in the train set: 1750, number of used features: 4
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.518857 -> initscore=0.075464
[LightGBM] [Info] Start training from score 0.075464
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[20]	valid_0's binary_logloss: 0.691162
🔁 Window 2500 — X_train: (1750, 8), X_test: (250, 8)
[LightGBM] [Info] Number of positive: 886, number of negative: 864
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.00014

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11

📁 Saved all artifacts for ORCL
🚫 The following tickers failed: ['GOOGL', 'AMZN', 'META', 'XOM', 'PEP', 'AVGO', 'PFE', 'CSCO', 'TMO', 'DHR', 'NEE', 'TXN', 'AMGN', 'UPS', 'PM']
